In [1]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

  1%|          | 81920/9912422 [00:00<00:12, 819087.52it/s]

9920512it [00:00, 28582723.80it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


32768it [00:00, 610180.80it/s]
  3%|▎         | 49152/1648877 [00:00<00:03, 477980.92it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:00, 10999978.05it/s]                         
8192it [00:00, 194120.62it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [3]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [5]:
model = Net()

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [9]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
for epoch in range(1, 10):
    train(epoch)
    test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.297996
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.300286
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.271068
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.245646
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.227338
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.178343
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.155018
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.015251
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.975394
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.760927
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.456537
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.258119
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.048184
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.965175
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.730890
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.705010
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.857119
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.741680
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.430011
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.635259
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.2094, Accuracy: 9401/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.312979
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.190979
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.233372
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.212463
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.086911
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.184105
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.175778
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.213113
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.111040
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.195877
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.216466
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.168498
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.192809
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.109306
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.213665
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.148788
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.106107
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.190279
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.076726
Train Epoch: 2 [12

KeyboardInterrupt: 